In [5]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [6]:
df = pd.read_csv('C:/langchain2/data_medical/generic_drug_combo.csv')

In [7]:
df

,output_text,intput_text
0,gastroesophageal reflux disease,what is the uses of Magaldrate
1,fever,what is the uses of Doxylamine-PE-DM-Acetamino...
2,"calcium carb-mag hydrox-simeth, alum-mag hydro...",which medication is used for flatulence
3,Bacterial Urinary Tract Infection,what is the uses of Cefepime
4,diclofenac sodium,which medication is used for gout
...,...,...
1365,"acetaminophen-pamabrom, ibuprofen",which medication is used for gout
1366,ampicillin-sulbactam,which medication is used for diverticulitis of...
1367,edema,what is the uses of Acetazolamide
1368,gastroesophageal reflux disease,what is the uses of Cimetidine Hcl


In [8]:
# # Get the maximum length of input_text column
# max_len = df['input_text'].apply(len).max()

# print('Maximum length of input_text column:', max_len)

In [9]:
# # count the number of rows in 'text' column containing more than 512 tokens
# count = N_df['input_text'].apply(lambda x: len(x.split()) > 600).sum()

# # print the count
# print(count)

In [10]:
from sklearn.model_selection import train_test_split
trn_df, val_df = train_test_split(df, test_size=0.1)
print('Length of Training Data: ', len(trn_df))
print('Length of Val Data: ', len(val_df))

Length of Training Data:  1233
Length of Val Data:  137


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import shutil
import argparse
import random
import numpy as np
import glob
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

tokenizer = AutoTokenizer.from_pretrained("C:/langchain2/data_medical/code_medical/model2")


In [12]:
# device = torch.device('cuda') 
# # check if CUDA is available
# if torch.cuda.is_available():
#     device = torch.device('cuda')
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())
# else:
#     # if CUDA is not available, exit gracefully
#     print("CUDA is not available. Exiting...")
#     exit()

In [13]:

# tokenizer = AutoTokenizer.from_pretrained("C:/langchain2/data_medical/code_medical/model2/checkpoint-71976")

# model = AutoModelForCausalLM.from_pretrained("C:/langchain2/data_medical/code_medical/model2/checkpoint-71976").to(device)

In [14]:
import os
import pickle
import logging
from transformers import PreTrainedTokenizer, MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from typing import Dict, List, Tuple

logger = logging.getLogger(__name__)

def construct_convo(row, tokenizer):
  # print(row)
  # print(tokenizer.encode(row[0]))
  # print(tokenizer.encode(row[-1]))
  conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row])) 
  print(conv) 
  flatten = lambda l: [item for sublist in l for item in sublist]
  conv = flatten(conv)
  # print(conv)
  return conv

class ConversationDataset(Dataset):
  def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):
    block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

    directory = args.cache_dir
    cached_features_file = os.path.join(
        directory, args.model_type + "_cached_lm_" + str(block_size)
    )

    if os.path.exists(cached_features_file) and not args.overwrite_cache:
      logger.info("Loading features from cached file %s", cached_features_file)
      with open(cached_features_file, "rb") as handle:
          self.examples = pickle.load(handle)
    else:
      logger.info("Creating features from dataset file at %s", directory)

      self.examples = []
      for _, row in df.iterrows():
          conv = construct_convo(row, tokenizer)
          self.examples.append(conv)
      
      logger.info("Saving features into cached file %s", cached_features_file)
      with open(cached_features_file, "wb") as handle:
        pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

In [15]:
# for _, row in trn_df.iterrows():
#   conv = construct_convo(row, tokenizer)
#   break

In [16]:

logger = logging.getLogger(__name__)

def construct_convo(row, tokenizer):
  conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row])) 
  flatten = lambda l: [item for sublist in l for item in sublist]
  conv = flatten(conv)
  return conv

class ConversationDataset(Dataset):
  def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=256):
    block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

    directory = args.cache_dir
    cached_features_file = os.path.join(
        directory, args.model_type + "_cached_lm_" + str(block_size)
    )

    if os.path.exists(cached_features_file) and not args.overwrite_cache:
      logger.info("Loading features from cached file %s", cached_features_file)
      with open(cached_features_file, "rb") as handle:
          self.examples = pickle.load(handle)
    else:
      logger.info("Creating features from dataset file at %s", directory)

      self.examples = []
      for _, row in df.iterrows():
          conv = construct_convo(row, tokenizer)
          self.examples.append(conv)
      
      logger.info("Saving features into cached file %s", cached_features_file)
      with open(cached_features_file, "wb") as handle:
        pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

In [17]:
# Caching and storing of data/checkpoints


def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
   return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

def set_seed(args):
   random.seed(args.seed)
   np.random.seed(args.seed)
   torch.manual_seed(args.seed)
   if args.n_gpu > 0:
       torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
   ordering_and_checkpoint_path = []


   glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))


   for path in glob_checkpoints:
       if use_mtime:
           ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
       else:
           regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
           if regex_match and regex_match.groups():
               ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))


   checkpoints_sorted = sorted(ordering_and_checkpoint_path)
   checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
   return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
   if not args.save_total_limit:
       return
   if args.save_total_limit <= 0:
       return


   # Check if we should delete older checkpoint(s)
   checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
   if len(checkpoints_sorted) <= args.save_total_limit:
       return

   number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
   checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
   for checkpoint in checkpoints_to_be_deleted:
       logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
       shutil.rmtree(checkpoint)

In [18]:
from transformers import PreTrainedModel, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm, trange

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )
    print("Batch size: ", args.train_batch_size)
    print("Data Loader length: ", len(train_dataloader))

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                print("Batch shape: ", batch.shape)
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [19]:
# Main runner
from transformers import AutoConfig

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)
        print(train_dataset)
        print("Train Dataset Length: ", len(train_dataset))
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [20]:
class Args():
    def __init__(self):
        self.output_dir = 'C:/langchain2/data_medical/code_medical/model2'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'C:/langchain2/data_medical/code_medical/model2'
        self.config_name = 'C:/langchain2/data_medical/code_medical/model2'
        self.tokenizer_name = 'C:/langchain2/data_medical/code_medical/model2'
        self.cache_dir = 'cached'
        self.block_size = 1024
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 1
        self.per_gpu_eval_batch_size = 1
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-6
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 10
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 5
        self.save_steps = 3000
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'


args = Args()

In [21]:
main(trn_df, val_df)

05/06/2023 16:49:50 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/06/2023 16:49:51 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x000002F33B222E90>
05/06/2023 16:49:51 - INFO - __main__ -   Creating features from dataset file at cached
05/06/2023 16:49:52 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_256
c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train Dataset Length:  1233
Batch size:  1
Data Loader length:  1233


05/06/2023 16:49:53 - INFO - __main__ -   ***** Running training *****
05/06/2023 16:49:53 - INFO - __main__ -     Num examples = 1233
05/06/2023 16:49:53 - INFO - __main__ -     Num Epochs = 10
05/06/2023 16:49:53 - INFO - __main__ -     Instantaneous batch size per GPU = 1
05/06/2023 16:49:53 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 1
05/06/2023 16:49:53 - INFO - __main__ -     Gradient Accumulation steps = 1
05/06/2023 16:49:53 - INFO - __main__ -     Total optimization steps = 12330
05/06/2023 16:49:53 - INFO - __main__ -     Starting fine-tuning.


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1233 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1233 [00:00<?, ?it/s]

05/06/2023 17:07:13 - INFO - __main__ -   Saving model checkpoint to C:/langchain2/data_medical/code_medical/model2\checkpoint-12000
05/06/2023 17:07:19 - INFO - __main__ -   Saving optimizer and scheduler states to C:/langchain2/data_medical/code_medical/model2\checkpoint-12000
05/06/2023 17:07:48 - INFO - __main__ -    global_step = 12330, average loss = 5.253072975615805
05/06/2023 17:07:48 - INFO - __main__ -   Saving model checkpoint to C:/langchain2/data_medical/code_medical/model2
05/06/2023 17:07:51 - INFO - __main__ -   Evaluate the following checkpoints: ['C:/langchain2/data_medical/code_medical/model2']
05/06/2023 17:07:53 - INFO - __main__ -   Creating features from dataset file at cached
05/06/2023 17:07:53 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_256
05/06/2023 17:07:53 - INFO - __main__ -   ***** Running evaluation  *****
05/06/2023 17:07:53 - INFO - __main__ -     Num examples = 137
05/06/2023 17:07:53 - INFO - __main__ -     Batch si

Evaluating:   0%|          | 0/137 [00:00<?, ?it/s]

05/06/2023 17:07:55 - INFO - __main__ -   ***** Eval results  *****
05/06/2023 17:07:55 - INFO - __main__ -     perplexity = tensor(146.4523)


{'perplexity_': tensor(146.4523)}

In [ ]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2Model


# Concatenate the text columns and replace '\n' with a space
df['input_text'] = df['BACKGROUND'].fillna('') + ' ' + df['OBJECTIVE'].fillna('') + ' ' + df['RESULTS'].fillna('') + ' ' + df['CONCLUSIONS'].fillna('')
df['input_text'] = df['input_text'].str.replace('\n', ' ')

# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Set the maximum length of each chunk
max_len = 512

# Process each row in the dataset
for index, row in df.iterrows():
    # Split the input text into chunks of fixed length
    input_text = row['input_text']
    chunks = [input_text[i:i+max_len] for i in range(0, len(input_text), max_len)]

    # Process each chunk separately and concatenate the output
    output = ''
    for chunk in chunks:
        input_ids = tokenizer.encode(chunk, return_tensors='pt')
        with torch.no_grad():
            model_output = model(input_ids)[0]
        output += tokenizer.decode(model_output[0, -max_len:], skip_special_tokens=True)

    # Store the output back into the dataframe
    df.at[index, 'output_text'] = output

# Save the updated dataframe
df.to_csv('your_output.csv', index=False)


In [ ]:


# Load the DataFrame
df = pd.read_csv("your_data.csv")

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenize the text and convert it to a TextDataset
tokenized_text = df.apply(lambda x: tokenizer.encode(x['BACKGROUND'] + x['OBJECTIVE'] + x['METHODS'] + x['RESULTS'] + x['CONCLUSIONS']), axis=1)
dataset = TextDataset(tokenized_text.tolist(), tokenizer=tokenizer)

# Set up the data collator and training arguments
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=1e-4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=500,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()
